In [1]:
import numpy as np
import pandas as pd
data_path="../data/ml-100k/u.data"
data_fields = ['user_id', 'item_id', 'rating', 'timestamp']
# all data file
data_df = pd.read_table(data_path, names=data_fields)

data_df.rating = (data_df.rating >= 3).astype(np.float32)
df=data_df

In [ ]:
对于每个user 弄出其t-basket的dict

In [6]:
df.groupby('user_id',as_index=False)['rating'].agg({"pos_cnt": 'sum'})

,user_id,item_id,rating,timestamp
0,196,242,1.0,881250949
1,186,302,1.0,891717742
2,22,377,0.0,878887116
3,244,51,0.0,880606923
4,166,346,0.0,886397596
...,...,...,...,...
99995,880,476,1.0,880175444
99996,716,204,1.0,879795543
99997,276,1090,0.0,874795795
99998,13,225,0.0,882399156


In [2]:
df=df.sort_values(['user_id','timestamp'])

In [44]:
user_basket={}
basket={0:[df.iloc[0].item_id]}
last_user=df.iloc[0].user_id
last_t=df.iloc[0].timestamp

In [48]:
basket

{0: [168.0, 172.0], 1: [165.0], 2: [156.0], 3: [196.0, 166.0], 4: [187.0]}

In [3]:
# 所有反馈为正反馈，无评分为负反馈 ->不太合理？测试集在训练时被当做负反馈了？——暂时这么做吧，可能需要剔除掉测试集的物品
# !!!!之所以放弃上面这种做法是因为上面这种做法测试集全是正样本 （考虑99个正样本，1个负样本 acc,prec,recall,F1,auc,ap之间的差异）

user_basket={}
basket={0:[df.iloc[0].item_id]}
last_user=df.iloc[0].user_id
last_t=df.iloc[0].timestamp
for i in range(df.shape[0]):
    if i ==0 or df.iloc[i].rating <=0:continue
    
    user = df.iloc[i].user_id
    if user!=last_user:
        last_user=user
        basket={}
    t = df.iloc[i].timestamp
#     print(i,t,last_t)
    if t ==last_t:
#         print('相同时间！')
        basket[len(basket)-1]+=[df.iloc[i].item_id]
    else:
        basket[len(basket)]=[df.iloc[i].item_id]
        last_t=t
    user_basket[user]=basket
    

In [20]:
res=[]
for i in user_basket:
    res+=[(i,j) for j in range(len(user_basket[i]))]
res=pd.DataFrame(res,columns=['user_id','t'])

In [28]:
res

,user_id,t
0,1.0,0
1,1.0,1
2,1.0,2
3,1.0,3
4,1.0,4
...,...,...
43515,943.0,82
43516,943.0,83
43517,943.0,84
43518,943.0,85


In [56]:
torch.tensor(0.).shape

torch.Size([])

In [57]:
torch.Tensor([0.]).shape

torch.Size([1])

In [42]:
import torch
x = torch.tensor([3,3,3,3])
y = x*x  #x.dot(x)
z = torch.mul(x,x)  #x.mul(x)
z = torch.mm(x,x)

print(y)
print(z)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [36]:
for idx,i in res.iterrows():
    print(i.t)
    if idx>5:
        break

0.0
1.0
2.0
3.0
4.0
5.0
6.0


In [30]:
user_basket[1][1]

[165.0]